# CIK

EDGAR assigns to filers a unique numerical identifier, known as a Central Index Key (CIK), when they sign up to make filings to the SEC. CIK numbers remain unique to the filer; they are not recycled.

Source: https://www.sec.gov/os/accessing-edgar-data 

# CIK, ticker, and exchange associations

Source 1: https://www.sec.gov/os/accessing-edgar-data

Source 2 (json): https://www.sec.gov/files/company_tickers.json

In [7]:
import pandas as pd    

df_CIK_ticker = pd.read_json('company_tickers.json')

df_CIK_ticker

,0,1,2,3,4,5,6,7,8,9,...,10655,10656,10657,10658,10659,10660,10661,10662,10663,10664
cik_str,320193,789019,1652044,1018724,1045810,1326801,1318605,1067983,1804583,1046179,...,1496254,1496254,1947559,1933644,1846702,1824204,1852039,748954,1299939,1923734
ticker,AAPL,MSFT,GOOGL,AMZN,NVDA,META,TSLA,BRK-B,RAASY,TSM,...,LTAFX,LTCFX,ANZGF,MDLVY,FIGP,PVOZ,FILG,ATLFF,CADE-PA,AFTWS
title,Apple Inc.,MICROSOFT CORP,Alphabet Inc.,AMAZON COM INC,NVIDIA CORP,"Meta Platforms, Inc.","Tesla, Inc.",BERKSHIRE HATHAWAY INC,Cloopen Group Holding Ltd,TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD,...,Alternative Strategies Income Fund,Alternative Strategies Income Fund,ANZ GROUP HOLDINGS LIMITED/ADR,"Medlive Technology Co., Ltd./ADR","Forge Group, Inc.",Park View OZ REIT Inc,Grayscale Filecoin Trust (FIL),ATLAS COPCO AB,Cadence Bank,Ark7 Properties Plus LLC


In [9]:
df_CIK_ticker = df_CIK_ticker.T
df_CIK_ticker.head()

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1652044,GOOGL,Alphabet Inc.
3,1018724,AMZN,AMAZON COM INC
4,1045810,NVDA,NVIDIA CORP


In [10]:
df_CIK_ticker.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10665 entries, 0 to 10664
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cik_str  10665 non-null  object
 1   ticker   10665 non-null  object
 2   title    10665 non-null  object
dtypes: object(3)
memory usage: 591.3+ KB


In [11]:
df_CIK_ticker.describe()

,cik_str,ticker,title
count,10665,10665,10665
unique,8250,10665,8249
top,1114446,AAPL,BARCLAYS BANK PLC
freq,27,1,27


Один CIK може відповідати одному чи більше тікеру. Наприклад:

Alternative Strategies Income Fund

Class A Shares (LTAFX)

Class C Shares (LTCFX)

https://www.sec.gov/ix?doc=/Archives/edgar/data/1496254/000158064223005248/alt-strat_486b.htm

=> тікер в нашому наборі даних має бути унікальним, але не CIK або title (назва компанії)

In [13]:
df_CIK_ticker[(df_CIK_ticker['cik_str'] == 1496254)]

,cik_str,ticker,title
10655,1496254,LTAFX,Alternative Strategies Income Fund
10656,1496254,LTCFX,Alternative Strategies Income Fund


In [14]:
from sqlalchemy import create_engine
import urllib

quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [15]:
df_CIK_ticker.to_sql('tb_CIK_ticker', schema='dbo', con = engine, if_exists='append',index=False)

In [16]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_CIK_ticker]')
result.fetchall()[0][0] 

10665